# Web-Mining - Assignment 3 (25 points total)

This **Home Assignment** is to be submitted and you will be given points for each of the tasks. It familiarizes you with the Apriori-algorithm, Association Rule Mining and Matrix factorization.
You can expect the following packages to be installed: numpy, pandas, matplotlib and of course the python standard library.

## Formalities
**Submit in a group of 3-4 people until 29.06.2021 23:59CET. The deadline is strict!**

## Evaluation and Grading
General advice for programming excercises at *CSSH*:
Evaluation of your submission is done semi automatically. Think of it as this notebook being 
executed once. Afterwards, some test functions are appended to this file and executed respectively.

Therefore:
* Submit valid _Python3_ code only!
* Use external libraries only when specified by task.
* Ensure your definitions (functions, classes, methods, variables) follow the specification if
  given. The concrete signature of e.g. a function usually can be inferred from task description, 
  code skeletons and test cases.
* Ensure the notebook does not rely on current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Double check if your code relies on presence of files or directories other than those mentioned
    in given tasks. Tests run under Linux, hence don't use Windows style paths 
    (`some\path`, `C:\another\path`). Also, use paths only that are relative to and within your
    working directory (OK: `some/path`, `./some/path`; NOT OK: `/home/alice/python`, 
    `../../python`).
* Keep your code idempotent! Running it or parts of it multiple times must not yield different
  results. Minimize usage of global variables.
* Ensure your code / notebook terminates in reasonable time.

**There's a story behind each of these points! Don't expect us to fix your stuff!**

Regarding the scores, you will get no points for a task if:
- your function throws an unexpected error (e.g. takes the wrong number of arguments)
- gets stuck in an infinite loop
- takes much much longer than expected (e.g. >1s to compute the mean of two numbers)
- does not produce the desired output (e.g. returns an descendingly sorted list even though we asked for ascending, returns the mean and the std even though we asked for only the mean, prints an output instead of returning it, ...)

In [1]:
# credentials of all team members (you may add or remove items from the dictionary)
team_members = [
    {
        'first_name': 'Pruthvi',
        'last_name': 'Hegde',
        'student_id': 404809
    },
    {
        'first_name': 'Mike',
        'last_name': 'Grüne',
        'student_id': 381076
    },
    {
        'first_name': 'Seyed Pouria',
        'last_name': 'Mirelmi',
        'student_id': 416910
    },
    {
        'first_name': 'Haron',
        'last_name': 'Nqiri',
        'student_id': 343289
    }
]

# Task 1: Apriori (18 points)

In this task you should implement a general version of the apriori algorithm.

The main idea of the apriori principle is, that if for a constraint you observe one property drops below a threshold (e.g. support drops below minimal support), it will never exceed that threshold for any more specific pattern.

Example for support:
$$ \sigma( \{\text{apple}, \text{banana}\}) < t \Rightarrow \sigma( \{\text{apple}, \text{banana}, \text{any_other_item}\}) < t$$

If any function $f$ fullfills

$$ f( \{\text{apple}, \text{banana}\}) < t \Rightarrow f( \{\text{apple}, \text{banana}, \text{any_other_item}\}) < t$$
 we call this property *anti-monotone*.

For this task a pattern will be a **sorted** tuple of items e.g. `('apple', 'banana', 'yoghurt')`. Items are just strings. Assume the database is given as a dataframe, each row representing a single transaction. It also has a column `'items'` that contains the items for that transaction as a set.

For an example database see below.

In [2]:
import pandas as pd
from collections import defaultdict
df = pd.read_csv ("weblog.csv", na_values="?")

# nice and short solution
def collect_websites(df) -> defaultdict(set):
    history = defaultdict(set)
    for user, website in zip(df.user_id, df.website_area):
        history[user].add(website)
    return pd.DataFrame.from_records(list(history.items()), columns=("user_id", "items") )
    return history

websites_db = collect_websites(df)

In [3]:
websites_db.head()

,user_id,items
0,10001,"{Support Desktop, End User Produced View, regwiz}"
1,10002,"{Support Desktop, Knowledge Base}"
2,10003,"{Support Desktop, Knowledge Base, Microsoft.co..."
3,10004,{Norway}
4,10005,{misc}


## a) Projecting a database (1)
Write a function `project_database(database, pattern)` that returns all rows of the `database` that contain all items in `pattern`. You can assume, that the database has a column `"items"`.

In [4]:
from typing import Tuple
import pandas as pd
from functools import partial

In [5]:
def project_database(database : pd.DataFrame, pattern : Tuple[str]) -> pd.DataFrame:
    ps = set(pattern)
    newDB = database.loc[database['items'].apply(lambda x: ps.issubset(x))]
    for index, row in newDB.iterrows():
        newDB.at[index,'items'] = tuple(sorted(row['items']))
    return newDB

In [6]:
# Example database
items = [{'apple', 'banana', 'milk'},
      {'apple', 'banana', 'citron'},
      {'apple', 'citron'},
      {'apple', 'citron'},
      {'apple',},
      {'milk',}]

prices = [10, 20, 13, 13, 5, 3]
example_db = pd.DataFrame.from_dict({'items':items, 'price' : prices})
example_db

,items,price
0,"{milk, apple, banana}",10
1,"{apple, banana, citron}",20
2,"{apple, citron}",13
3,"{apple, citron}",13
4,{apple},5
5,{milk},3


In [7]:
display(project_database(example_db, tuple()))
display(project_database(example_db, ("apple", "citron")))
display(project_database(example_db, ("apple", )))# notice comma
display(project_database(example_db, ("apple", "banana", "citron")))




#	items 	price
#0 	(apple, banana, milk) 	10
#1 	(apple, banana, citron) 	20
#2 	(apple, citron) 	13
#3 	(apple, citron) 	13
#4 	(apple,) 	5
#5 	(milk,) 	3

#	items 	price
#1 	(apple, banana, citron) 	20
#2 	(apple, citron) 	13
#3 	(apple, citron) 	13

#	items 	price
#0 	(apple, banana, milk) 	10
#1 	(apple, banana, citron) 	20
#2 	(apple, citron) 	13
#3 	(apple, citron) 	13
#4 	(apple,) 	5

#	items 	price
#1 	(apple, banana, citron) 	20

,items,price
0,"(apple, banana, milk)",10
1,"(apple, banana, citron)",20
2,"(apple, citron)",13
3,"(apple, citron)",13
4,"(apple,)",5
5,"(milk,)",3


,items,price
1,"(apple, banana, citron)",20
2,"(apple, citron)",13
3,"(apple, citron)",13


,items,price
0,"(apple, banana, milk)",10
1,"(apple, banana, citron)",20
2,"(apple, citron)",13
3,"(apple, citron)",13
4,"(apple,)",5


,items,price
1,"(apple, banana, citron)",20


## b) Calculating the support/average price (1+1)
Write a function `calc_support(database, pattern)` that computes the support for the `pattern` in the `database`. Thereby use the `project_database` function defined in a).

Write another function `calc_average_price(database, pattern)` that calculates the average price for the pattern.


Make sure you are using the function defined in a). Notice that `calc_average_price` is **not** anti-monotone

In [8]:
import pandas as pd
from typing import Tuple, List, Iterable
def calc_support(database : pd.DataFrame, pattern : Tuple[str]) -> float:
    no_of_transactions = len(database.index)

    proj_database = project_database(database,pattern)
    no_of_matched_transactions = len(proj_database.index)
    support = no_of_matched_transactions/no_of_transactions
    return support
    
    
def calc_average_price(database : pd.DataFrame, pattern : Tuple[str]) -> float:
    proj_database = project_database(database,pattern)
    price = proj_database['price'].values
    total = 0
    for i in price:
        total = total + int(i)
    avg = total / len(price)
    return avg

In [9]:
print(calc_support(example_db, tuple()))
print(calc_support(example_db, ("apple", "citron")))
print(calc_support(example_db, ("apple", )))# notice comma
print(calc_support(example_db, ("apple", "banana", "citron")))

#1.0
#0.5
#0.8333333333333334
#0.16666666666666666

1.0
0.5
0.8333333333333334
0.16666666666666666


In [10]:
print(calc_average_price(example_db, tuple()))
print(calc_average_price(example_db, ("apple", "citron")))
print(calc_average_price(example_db, ("apple", )))# notice comma
print(calc_average_price(example_db, ("apple", "banana", "citron")))

#10.666666666666666
#15.333333333333334
#12.2
#20.0

10.666666666666666
15.333333333333334
12.2
20.0


## c) Generating initial candidates (1+2)
We want to minimize the number of times that we need to scan the database (or in our case to "project" the database) as much as possible. Therefore, we want to use the apriori principle as much as possible.

To that extent, first generate *initial candidates* of length $l+1$ from all the frequent patterns of length $l$ (this task) which are then further pruned by using the apriori principle (next task).

#### a) The simple way (1)
The easy way is to collect all items that are in all the patterns and consider all combinations of those items of length $l+1$. It has runtime $O(\binom{I}{l+1})$. $\binom{x}{y}$ meaning the binomial coeff. $I$ is the number of items.


#### b) The faster way (2)
You can gain significant speedup in this step by only generating initial candidates of length $l+1$ that arise when matching pairs of frequent patterns where the first $l-1$ items are the same. Implement this strategy as well.


In [11]:
import itertools

In [12]:
def getSubTuples(itemset, n):
    return itertools.combinations(itemset, n)

def generate_patterns_a(pattern: List[Tuple[str]]) -> Iterable[Tuple[str]]:
    items = set()
    if (len(pattern) != 0):
        for i in pattern:
            l = len(i) + 1
            for j in i:
                items.add(j)

        subsets = getSubTuples(sorted(items), l)
    else:
        subsets = []
    return subsets

In [13]:
# example usage
generate_patterns_a([("a", "b"), ("a", "c"), ("c", "d")])
# <generator object generate_patterns_a at 0x00000...
# <itertools.combinations at 0x1...
# Could be anything that is iterable

In [14]:
# example usage
# notice the items in the patterns are still in correct order
print("#", list(generate_patterns_a([("a", "b"), ("a", "c"), ("c", "d")])))
print("#", list(generate_patterns_a([("a", "b"), ("a", "c"), ("c", "d"), ("c", "e")])))
print("#", list(generate_patterns_a([])))
print("#", list(generate_patterns_a([("a", "b"), ("a", "c"), ("c", "d"), ("c", "e"), ("a", "d")])))
# [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'c', 'd'), ('b', 'c', 'd')]
# [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'b', 'e'), ('a', 'c', 'd'), ('a', 'c', 'e'), ('a', 'd', 'e'), ('b', 'c', 'd'), ('b', 'c', 'e'), ('b', 'd', 'e'), ('c', 'd', 'e')]
# []
# [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'b', 'e'), ('a', 'c', 'd'), ('a', 'c', 'e'), ('a', 'd', 'e'), ('b', 'c', 'd'), ('b', 'c', 'e'), ('b', 'd', 'e'), ('c', 'd', 'e')]

# [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'c', 'd'), ('b', 'c', 'd')]
# [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'b', 'e'), ('a', 'c', 'd'), ('a', 'c', 'e'), ('a', 'd', 'e'), ('b', 'c', 'd'), ('b', 'c', 'e'), ('b', 'd', 'e'), ('c', 'd', 'e')]
# []
# [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'b', 'e'), ('a', 'c', 'd'), ('a', 'c', 'e'), ('a', 'd', 'e'), ('b', 'c', 'd'), ('b', 'c', 'e'), ('b', 'd', 'e'), ('c', 'd', 'e')]


In [15]:
def generate_patterns_b(pattern : List[Tuple[str]]) -> List[Tuple[str]]:
    s = set()
    if len(pattern) != 0 :
        l = len(pattern)
        for i in range(l):
            for j in range(i+1, l):
                item_set = set()
                iden = True
                for r in range(len(pattern[i])-1):
                    if pattern[i][r] != pattern[j][r]:
                        iden = False
                        continue
                if iden is True:
                    item_set.update(pattern[i]+pattern[j])
                    s.add(tuple(sorted(item_set)))
        li = list(s)
        li.sort(key=lambda tup: tup[1])
    else:
        li = []
    return li
                    


In [16]:
# example usage
# Hope this already makes clear why this is the better approach
print("#", list(generate_patterns_b([("a", "b"), ("a", "c"), ("c", "d")])))
print("#", list(generate_patterns_b([("a", "b"), ("a", "c"), ("c", "d"), ("c", "e")])))
print("#", list(generate_patterns_b([])))
print("#", list(generate_patterns_b([("a", "b"), ("a", "c"), ("c", "d"), ("c", "e"), ("a", "d")])))
# [('a', 'b', 'c')]
# [('a', 'b', 'c'), ('c', 'd', 'e')]
# []
# [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'c', 'd'), ('c', 'd', 'e')]

# [('a', 'b', 'c')]
# [('a', 'b', 'c'), ('c', 'd', 'e')]
# []
# [('a', 'b', 'c'), ('a', 'b', 'd'), ('a', 'c', 'd'), ('c', 'd', 'e')]


## d) Generate next level candidates (2)

We provide a function `generate_next_level_candidates` that takes a list of patterns (assume that the constraint holds for all the patterns that are supplied) and a list of candidate patterns. It returns all the candidates that need to be checked against the database. There is one part of this function missing, the function `prune_candidates`. This function should further prune the *initial candidates* obtained in the previous step.

The function `prune_candidates` `yield`s all patterns of length `l+1` from the candidate patterns that are not already excluded through the apriori principle. I.e. a candidate of length $l+1$ is returned if **all** sub-patterns of length `l` are in `patterns`.

In [17]:
def prune_candidates(patterns : List[Tuple[str]], candidates : Iterable[Tuple[str]]) -> Iterable[Tuple[str]]:
    # Notice, that the length of candidates might not be available
    if len(patterns) != 0:
        s = set()
        for can in candidates:
            i = True
            for sub in getSubTuples(can, len(can) - 1):
                if sub not in patterns:
                    i = False
            if i is True:
                s.add(can)
        li = list(s)
        yield from li
    else:
        yield from ()

In [18]:
print("#", list(prune_candidates([("a",), ("b",), ("c",)], [("a", "b"), ("b", "c"), ("e", "f")])))
print("#", list(prune_candidates([("a", "b"), ("a", "c"), ("c", "d")], [("a", "b", "c")])))
print("#", list(prune_candidates([("a", "b"), ("a", "c"), ("b", "c")], [("a", "b", "c")])))
print("#", list(prune_candidates([("a", "b", "c"), ("a", "b", "d"), ("a", "c", "d")], [("a", "b", "c", "d")])))
print("#", list(prune_candidates([("a", "b", "c"), ("a", "b", "d"), ("a", "c", "d"), ("b", "c", "d")], [("a", "b", "c", "d")])))

# [('a', 'b'), ('b', 'c')]
# []
# [('a', 'b', 'c')]
# []
# [('a', 'b', 'c', 'd')]

# [('a', 'b'), ('b', 'c')]
# []
# [('a', 'b', 'c')]
# []
# [('a', 'b', 'c', 'd')]


In [19]:
# !!! IMPORTANT !!!
generate_patterns = generate_patterns_b
# do not forget to change to _a if you don't manage generate_patterns_b

In [20]:
def generate_next_level_candidates(patterns : List[Tuple[str]]) -> Iterable[Tuple[str]]:
    if len(patterns) == 0:
        return []
    level = len(patterns[0])
    
    resulting_patterns = []
    candidates = generate_patterns(patterns)
    return prune_candidates(patterns,  candidates)

In [21]:
print("#", list(generate_next_level_candidates([("a",), ("b",), ("c",)])))
print("#", list(generate_next_level_candidates([("a", "b"), ("a", "c"), ("c", "d")])))
print("#", list(generate_next_level_candidates([("a", "b", "c"), ("a", "b", "d"), ("a", "c", "d")])))
print("#", list(generate_next_level_candidates([("a", "b", "c"), ("a", "b", "d"), ("a", "c", "d"), ("b", "c", "d")])))
# [('a', 'b'), ('a', 'c'), ('b', 'c')]
# []
# []
# [('a', 'b', 'c', 'd')]

# [('a', 'c'), ('a', 'b'), ('b', 'c')]
# []
# []
# [('a', 'b', 'c', 'd')]




## e) Bringing general apriori together (4)

Write a function `generic_apriori` that returns all patterns with length of at most (<=) `max_depth` for which `calc_value` is at least (>=) `t` on that database. Assume that the `calc_value` function is anti-monotone. Use the apriori principle to reduce the number of calls to the `calc_value` functions. The function returns a dictionary which maps all patterns for which `calc_value` exceeds as keys and maps them onto their corresponding value of `calc_value`.

Assume that the database is given as a pandas DataFrame with column `'items'` which contains all the itemsets. There might be other columns which are used by the constraint. The maximum depth restricts the number of items in a pattern.

In [22]:
from typing import Dict, Callable
def generic_apriori(calc_value : Callable[[pd.DataFrame, Tuple[str]], float],
                    t : float,
                    items : List[Tuple[str]],
                    max_depth : int,
                    database : pd.DataFrame) -> Dict[Tuple[str], float]:
    s = set()
    d = dict()
    for item in items:
        s.update(getSubTuples(item, 1))
    for item in s:
        val = calc_value(database, item)
        if val >= t:
            d[item] = val
    s = set(d.keys())
    i = 2
    while i <= max_depth:
        s = set(prune_candidates(s, generate_next_level_candidates(list(s))))
        for item in s:
            val = calc_value(database, item)
            if val >= t:
                d[item] = val
        s = set(item for item in d.keys() if len(item) == i)
        i += 1
    return d

## f) Classic apriori (2)

Write function `fim_apriori` that performs classic frequent-itemset mining (FIM) using the apriori principle. `min_supp` is a float between 0 and 1. It is used as a lower bound (>=) for the support. The function returns all patterns that appear at least `min_supp` times and are of at most `max_depth` depth. You should use the `generic_apriori` function.

Thereby use the `calc_support` function from b)

In [23]:
def fim_apriori(database : pd.DataFrame, max_depth : int, min_supp : float) -> List[Tuple[str]]:
    assert min_supp >=0 and  min_supp<= 1
    d = generic_apriori(calc_support, min_supp, database['items'].values, max_depth, database)
    return d

In [24]:
# support for 1 itemsets
# apple  : 5/6
# banana : 2/6
# citron : 3/6
# milk   : 1/6
print(fim_apriori(example_db, 2, 0.5)) # {('citron',): 0.5, ('apple',): 0.8333333333333334, ('apple', 'citron'): 0.5}
print(fim_apriori(example_db, 1, 0.7)) # {('apple',): 0.8333333333333334}
print(fim_apriori(example_db, 1, 0.4)) # {('citron',): 0.5, ('apple',): 0.8333333333333334}
print()

{('citron',): 0.5, ('apple',): 0.8333333333333334, ('apple', 'citron'): 0.5}
{('apple',): 0.8333333333333334}
{('citron',): 0.5, ('apple',): 0.8333333333333334}



In [25]:
# used to identify the number of calls to min_supp
from contextlib import contextmanager
@contextmanager
def my_mock(func, replace):
    """If a function references another function it can be replaced with this function"""
    tmp = []
    for key, value in replace.items():
        current_func=func
        splits = key.split(".")
        for func_name in splits[:-1]:
            current_func = current_func.__globals__[func_name]
        tmp.append(current_func.__globals__[splits[-1]])
        current_func.__globals__[splits[-1]]   = value
    try:
        yield
    finally:
        for key, value in zip(replace.keys(), tmp):
            func.__globals__[key] = value

In [26]:
# the following class can be used instead of the calc_support function
# it keeps track of the calls that have been made to it
class MinSupport_CountCalls:
    def __init__(self):
        self.number_of_calls = 0
        self.calc_support = calc_support
        self.called_for_patterns=set()
    def __call__(self, database, pattern):
        self.number_of_calls +=1
        self.called_for_patterns.add(pattern)
        return self.calc_support(database, pattern)

count_obj = MinSupport_CountCalls()
with my_mock(fim_apriori, {'calc_support' : count_obj}):
    fim_apriori(example_db, 2, 0.5)
    print(count_obj.number_of_calls) # support was calculated 5 times
    print(count_obj.called_for_patterns) # called with these arguments
    # {('apple', 'citron'), ('banana',), ('apple',), ('milk',), ('citron',)}
    

5
{('apple', 'citron'), ('banana',), ('citron',), ('apple',), ('milk',)}


In [27]:
count_obj = MinSupport_CountCalls()
with my_mock(fim_apriori, {'calc_support' : count_obj}):
    fim_apriori(example_db, 2, 0.3)
    print(count_obj.number_of_calls) # support was calculated 10 times
    print(count_obj.called_for_patterns) # called with these arguments:
    #{('banana', 'citron'), ('apple', 'citron'), ('banana',), ('apple',), ('apple', 'banana'), ('milk',), ('banana', 'milk'), ('apple', 'milk'), ('citron',), ('citron', 'milk')}

10
{('banana', 'milk'), ('apple', 'banana'), ('apple', 'citron'), ('banana',), ('apple', 'milk'), ('citron', 'milk'), ('citron',), ('apple',), ('milk',), ('banana', 'citron')}


In [28]:
%%time
print(fim_apriori(websites_db, 2, 0.3))
#{('Free Downloads',): 0.3312442678080098}
# Wall time: 5.29 s

{('Free Downloads',): 0.3312442678080098}
CPU times: user 9.46 s, sys: 7.47 ms, total: 9.47 s
Wall time: 9.62 s


In [29]:
%%time
print(fim_apriori(websites_db, 5, 0.25))
#{('Microsoft.com Search',): 0.2587282176704372, ('Internet Explorer',): 0.2868541730357689, ('Free Downloads',): 0.3312442678080098}
#Wall time: 5.58 s

{('Free Downloads',): 0.3312442678080098, ('Internet Explorer',): 0.2868541730357689, ('Microsoft.com Search',): 0.2587282176704372}
CPU times: user 10.1 s, sys: 19.9 ms, total: 10.1 s
Wall time: 10.2 s


In [30]:
%%time
print(fim_apriori(websites_db, 5, 5260/len(websites_db)))
# {('Internet Explorer',): 0.2868541730357689,
# ('Microsoft.com Search',): 0.2587282176704372,
# ('isapi',): 0.16294711097523693,
# ('Free Downloads',): 0.3312442678080098,
# (Free Downloads', 'Internet Explorer'): 0.16080709263222256}
#Wall time: 5.65 s

{('Free Downloads',): 0.3312442678080098, ('Internet Explorer',): 0.2868541730357689, ('isapi',): 0.16294711097523693, ('Microsoft.com Search',): 0.2587282176704372, ('Free Downloads', 'Internet Explorer'): 0.16080709263222256}
CPU times: user 10.5 s, sys: 20.1 ms, total: 10.6 s
Wall time: 10.7 s


## f) Association rule miner (4)

Write a function `assoc_miner` that uses `fim_apriori` and `generic_apriori` to find all association rules with at least `min_supp` and `min_conf`. It threrefore first mines all `min_supp` frequent itemsets and for each it finds the `min_conf` association rules. Use the apriori principle to mine all the association rules from a pattern.

The most difficult part of this task is to find the right parameters which are passed to the `generic_apriori` function which then returns for a pattern $Z$ all association rules $A \Rightarrow B$ with $A\cup B = Z$ and confidence of at least (>=) `min_conf`.
In the lecture it was presented, that the confidence of a single association rule $Z$ has a downward closure property with respect to the consequent $B$ of an association rule $A\Rightarrow B$ with $Z=A\cup B$. You can thus use the `generic_apriori` implementation with a custom `calc_value` function to prune the search space for association rules which come from a single itemset $Z$.
To that extend you might want to write a function that computes the antecedent/consequence $ A = Z\setminus B$ given the consequent/antecedent $B$.
You might also want to use the `functools.partial` function because the `calc_value` function is different for each pattern $Z$.
Also notice that you do not really need to query the database any more as, if $Z$ is frequent, all its subsets are also frequent i.e. the support was already calculated by the initial call to `fim_apriori`.

In [31]:
def assoc_miner(database : pd.DataFrame,
                max_depth : int,
                min_supp : float,
                min_conf : float) -> pd.DataFrame:
    tups = fim_apriori(database, max_depth, min_supp)
    
    return pd.DataFrame.from_records([], columns=["antecedent", "consequence", "support", "confidence"])

In [32]:
with pd.option_context('display.max_rows', 300, 'display.max_columns', 7, 'display.expand_frame_repr', False):
    print(assoc_miner(example_db, 3, 0.3, 0.01))
#      antecedent consequence   support  confidence
#0  (banana,)    (apple,)  0.333333         1.0
#1   (apple,)   (banana,)  0.333333         0.4
#2  (citron,)    (apple,)  0.500000         1.0
#3   (apple,)   (citron,)  0.500000         0.6


    print(assoc_miner(example_db, 4, 0.1, 0.6))
#             antecedent consequence   support  confidence
#0         (banana,)    (apple,)  0.333333         1.0
#1         (citron,)    (apple,)  0.500000         1.0
#2          (apple,)   (citron,)  0.500000         0.6
#3  (banana, citron)    (apple,)  0.166667         1.0
#4    (banana, milk)    (apple,)  0.166667         1.0
#5     (apple, milk)   (banana,)  0.166667         1.0

Empty DataFrame
Columns: [antecedent, consequence, support, confidence]
Index: []
Empty DataFrame
Columns: [antecedent, consequence, support, confidence]
Index: []


In [33]:
%%time
result = assoc_miner(websites_db, 4, 1000/len(websites_db), 0.6)
display(result)
with pd.option_context('display.max_rows', 300, 'display.max_columns', 7, 'display.expand_frame_repr', False):
    print(str(result))
    
#                          antecedent               consequence   support  confidence
#0                  (Knowledge Base,)        (Support Desktop,)  0.055212    0.608491
#1                      (Windows 95,)  (Windows Family of OSs,)  0.032437    0.914655
#2               (Windows95 Support,)                  (isapi,)  0.046072    0.841429
#3     (Internet Explorer, Products )         (Free Downloads,)  0.031733    0.670110
#4            (Knowledge Base, isapi)        (Support Desktop,)  0.033231    0.708605
#5  (Knowledge Base, Support Desktop)                  (isapi,)  0.033231    0.601883


,antecedent,consequence,support,confidence


Empty DataFrame
Columns: [antecedent, consequence, support, confidence]
Index: []
CPU times: user 18.2 s, sys: 20 ms, total: 18.2 s
Wall time: 18.4 s


# Task 2: Matrix factorization (7 points)

In this task you should implement matrix factorization using stochastic gradient descent. The goal is to find matrices $P$ and $Q$ given a matrix $R$ such that $P \cdot Q\approx R$

Unlike in "normal" matrix decomposition, the rating matrix $R$ can contain blank entries (given as 0). The factorization should be trained only on **non-blank entries**. Use squared error as your loss function and stochastic gradient descent (SGD) for optimization.


The matrices for K latent factors are of size

$$
P: (I, K)\\
Q: (K, J)\\
R: (I, J)
$$
The Loss for one entry is $L_{i,j} = (R_{i,j} - \sum_{k} P_{i,k} \cdot Q_{k,j})^2$.
The average loss for a set of index pairs is $I$
$$
L_I = \frac{1}{|I|}\sum_{(i,j) \in I} L_{i,j}
$$
The partial derivatives of the loss are
$$
\frac{dL_{i,j}}{dP_{i,v}} = -2(R_{i, j} - \sum_{k} P_{i,k} \cdot Q_{k,j})Q_{v,j} \\
\frac{dL_{i,j}}{dQ_{v,j}} = -2(R_{i, j} - \sum_{k} P_{i,k} \cdot Q_{k,j})P_{i,v}
$$
SGD update for observation $i,j$

$P_{i,v} = P_{i,v} - \alpha \frac{dL_{i,j}}{dP_{i,v}}$

$Q_{v,j} = Q_{v,j} - \alpha \frac{dL_{i,j}}{dQ_{v,j}}$

When performing SGD, compute all the partial derivates for one index pair $i,j$ first and then update the values.
For testing, consider the utility matrix in the file 'utility_matrix.csv'.

## a) Average Loss function (1)
Write a loss function `average_loss(R, P, Q, ijs=None)` that computes the average reconstruction loss of your matrix factorization. If `ijs` are supplied only for those indices loss is calculated, otherwise for the entire matrix. `ijs` is list of tuples of indices.



In [34]:
import numpy as np
from numpy.random import default_rng

In [35]:
def average_loss(R, P, Q, ijs=None):
    if ijs is None:
        ijs = [(i,j) for i,_ in enumerate(R) for j,_ in enumerate(R[i])]
    L = 0
    for i, j in ijs:
        L += (R[i][j]-sum([P[i,k]*Q[k][j] for k,_ in enumerate(Q)]))**2
    return L/len(ijs)

In [36]:
# example usage
rng=default_rng(1)
n=4
m=5
k=2
P = rng.random((m,k))
Q = rng.random((k,n))
R = rng.random((m,n))
average_loss(R,P,Q) 
# should be close to
# 0.14863680056176592

0.14863680056176592

In [37]:
# example usage
rng=default_rng(2)
n=10
m=12
k=2
P = rng.random((m,k))
Q = rng.random((k,n))
R = rng.random((m,n))
average_loss(R,P,Q)
# should be close to
# 0.1682872357391108

0.1682872357391108

In [38]:
# example usage
rng=default_rng(3)
n=10
m=12
k=2
ijs = [(1,3), (4,5), (3,4)]
P = rng.random((m,k))
Q = rng.random((k,n))
R = rng.random((m,n))
average_loss(R,P,Q, ijs=ijs)
# should be close to
# 0.22473964744830408

0.22473964744830402

## b) Factorize (2)

Implement matrix factorization from scratch by implementing a function `matrix_factorization` with

* R: Matrix of given ratings (numpy array)
* k: Number of latent factors (int)
* alpha: learning rate (float)
* rng: random number generator (numpy.random.default_rng)
* epochs: the number of epoch to train (integer)
* ijs_train: indices to train on (list of tuples of two integers)
* ijs_test: indices to test on (list of tuples of two integers)

It returns the P, Q matrix, train-loss-list and test-loss-list. The loss is calculated after each epoch of training and is then appended to the list. One epoch is performing an update for all the `ijs_train`. Leave the `test_loss_list` empty if `ijs_test` are not specified.


In [39]:
def matrix_factorization(R : np.array,
                         K : int,
                         alpha : float,
                         rng,
                         epochs : int,
                         ijs_train : List[Tuple[int]],
                         ijs_test : List[Tuple[int]]=None):
    #randomly initialize matrices
    U,I = R.shape
    P = rng.random((U,K))
    Q = rng.random((K,I))
    
    train_losses = []
    test_losses = []
    for _ in range(epochs):
        for i,j in ijs_train:
            grad_P = [-2*(R[i][j]-sum([P[i,k]*Q[k][j] for k,_ in enumerate(Q)]))*Q[v][j] for v,_ in enumerate(Q)]
            grad_Q = [-2*(R[i][j]-sum([P[i,k]*Q[k][j] for k,_ in enumerate(Q)]))*P[i][v] for v,_ in enumerate(P[i])]
            for v,_ in enumerate(Q):
                P[i][v] = P[i][v] - alpha*grad_P[v]
                Q[v][j] = Q[v][j] - alpha*grad_Q[v]
        train_losses.append(average_loss(R,P,Q, ijs_train))
        if ijs_test is not None:
            test_losses.append(average_loss(R,P,Q, ijs_test))
    return P, Q, train_losses, test_losses

In [40]:
# Generate example:
R = np.genfromtxt('utility_matrix.csv', delimiter=',',dtype=np.float64)
rng=default_rng(1)
ijs = [(i,j) for i, j in zip(*np.nonzero(R))]
epochs=2
P, Q, l_train,l_test = matrix_factorization(R, 4, 0.001, rng, epochs, ijs)
print(l_train)

[6.12028559684065, 5.702120509768838]


```python
# epochs = 1
# P
array([[0.53030017, 0.96568957, 0.17039495, 0.96478729],
       [0.32009701, 0.43594586, 0.83751995, 0.41845338],
       [0.56471082, 0.03854926, 0.76744862, 0.54845707],
       [0.35681508, 0.80943855, 0.33222907, 0.47213126],
       [0.14975727, 0.41927103, 0.22232818, 0.2733064 ],
       [0.7632887 , 0.29660883, 0.50014336, 0.99427529],
       [0.97469191, 0.74041843, 0.55915683, 0.29096398],
       [0.17618862, 0.98158305, 0.53054081, 0.12794982],
       [0.62839181, 0.78786586, 0.61887645, 0.92215325],
       [0.05183058, 0.54145109, 0.47416302, 0.07551933],
       [0.65114207, 0.86260615, 0.60851652, 0.27028487],
       [0.8600531 , 0.52663855, 0.53355849, 0.76868257],
       [0.15773973, 0.82380165, 0.69349028, 0.79731367],
       [0.20945627, 0.82255223, 0.21504987, 0.09967779],
       [0.86684443, 0.87304182, 0.89573504, 0.48592394],
       [0.29209354, 0.0288247 , 0.67087993, 0.73952057],
       [0.8405461 , 0.28475842, 0.22651716, 0.64898595],
       [0.81803697, 0.97569678, 0.1701876 , 0.49792483],
       [0.90125284, 0.42775465, 0.59711584, 0.03434974],
       [0.68053857, 0.93020444, 0.83722127, 0.89347768]])
# Q.T
array([[0.67905308, 0.71703279, 0.6783386 , 0.28217364],
       [0.28671191, 0.2245397 , 0.46818009, 0.11601512],
       [0.79664329, 0.43092275, 0.8926013 , 0.28903537],
       [0.24819143, 0.041076  , 0.66335624, 0.79366653],
       [0.85183337, 0.29365763, 0.83269148, 0.71732725],
       [0.10102076, 0.46937421, 0.37185553, 0.15782279],
       [0.85353666, 0.1410099 , 0.56891687, 0.40167918],
       [0.20774953, 0.67103893, 0.24104121, 0.46453524],
       [0.39950197, 0.41217316, 1.01234667, 0.68805916],
       [0.36029593, 0.78600697, 0.28899546, 0.50123434]])
# l_train
 [6.120285596840649]
# l_test
 []
```

```python
# epochs = 2
# P
array([[0.54898348, 0.98157321, 0.19648697, 0.98124088],
       [0.32886539, 0.44907869, 0.84768277, 0.42812938],
       [0.58012071, 0.05003125, 0.78152081, 0.55921641],
       [0.38441392, 0.83141728, 0.3615508 , 0.49147467],
       [0.1662158 , 0.43662063, 0.24186757, 0.2852157 ],
       [0.77648636, 0.31330686, 0.51525653, 1.00812756],
       [0.98826968, 0.75669342, 0.57721966, 0.30538718],
       [0.19191371, 0.99378748, 0.54508121, 0.14019524],
       [0.63329215, 0.79907542, 0.62449599, 0.92703749],
       [0.0645117 , 0.55504459, 0.48935008, 0.08918713],
       [0.66116901, 0.8730181 , 0.62392472, 0.28066017],
       [0.88065328, 0.54456397, 0.55651066, 0.78492335],
       [0.16776704, 0.82830911, 0.70377071, 0.80760509],
       [0.22800606, 0.84385433, 0.23923373, 0.11851386],
       [0.87842173, 0.88485997, 0.91440644, 0.49974584],
       [0.31099925, 0.0515082 , 0.69662905, 0.75999372],
       [0.84561165, 0.2879153 , 0.23761533, 0.65868291],
       [0.83158875, 0.98837475, 0.19004603, 0.51386302],
       [0.90773606, 0.43283726, 0.604502  , 0.04412471],
       [0.6871331 , 0.94118386, 0.84689142, 0.90100045]])
# Q.T
array([[0.69701391, 0.74199718, 0.70229101, 0.30657812],
       [0.32711821, 0.26978508, 0.50493145, 0.15823725],
       [0.82398116, 0.46445741, 0.91741003, 0.31943684],
       [0.28343279, 0.07483107, 0.69377651, 0.82312758],
       [0.87141708, 0.32355552, 0.8543836 , 0.73583079],
       [0.13855796, 0.51664766, 0.40165922, 0.18632909],
       [0.88099643, 0.17496533, 0.59388225, 0.42629548],
       [0.24995286, 0.70774974, 0.28510039, 0.50711432],
       [0.42283823, 0.44255497, 1.02792469, 0.71036147],
       [0.40242657, 0.84484156, 0.33390436, 0.54527132]])
# l_train
[6.120285596840649, 5.702120509768838]
# l_test
[]
```

## c) Evaluate the training (1+1+2)

Evaluate the training by visualising the training- and test-loss as a function of epochs for different values of `alpha`. Apply it to the 'utility_matrix.csv'.  Perform the train test split like so (it's roughly 80/20):

```python
R = np.genfromtxt('utility_matrix.csv', delimiter=',',dtype=np.float)
rng = default_rng(13)
ijs = [(i,j) for i, j in zip(*np.nonzero(R))]
rng.shuffle(ijs)
ijs_train=ijs[:122]
ijs_test=ijs[122:]
```

The different values for `alpha` are `[0.05, 0.01, 0.001, 0.0001]`.
Use K=5 latent factors. Train for 20 and 500 epochs.
Save your plot as `training20.png` and `training500.png`.
Save a description + explanation of what is happening as `training.txt`. Write 4+ sentences.

You will get 1 point for the plots each and two points for the text.

In [41]:
import matplotlib.pyplot as plt

In [42]:
#loss_dict = {}
R = np.genfromtxt('utility_matrix.csv', delimiter=',',dtype=np.float)
rng=default_rng(13)
ijs = [(i,j) for i, j in zip(*np.nonzero(R))]
rng.shuffle(ijs)
ijs_train=ijs[:122]
ijs_test=ijs[122:]

In [43]:
fig_loss = plt.figure(figsize=(12, 12))
for a in [0.05, 0.01, 0.001, 0.0001]:
    P, Q, l_train,l_test = matrix_factorization(R, 5, a, rng, 20, ijs_train, ijs_test)
    plt.plot(l_train, label=f"Train Loss a ={a}")
    plt.plot(l_test, label=f"Test Loss a ={a}")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xticks(np.arange(len(l_test)), list(map(lambda x: str(x+1), np.arange(len(l_test)))))
plt.legend()
plt.savefig("training20.png")
plt.close()

In [44]:
fig_loss = plt.figure(figsize=(32, 32))
for a in [0.05, 0.01, 0.001, 0.0001]:
    P, Q, l_train,l_test = matrix_factorization(R, 5, a, rng, 500, ijs_train, ijs_test)
    plt.plot(l_train, label=f"Train Loss a ={a}")
    plt.plot(l_test, label=f"Test Loss a ={a}")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xticks(np.arange(len(l_test)), list(map(lambda x: str(x+1) if (x+1) % 10 == 0 else "", np.arange(len(l_test)))))
plt.legend()
plt.savefig("training500.png")
plt.close()

In [45]:
with open("training.txt", "w") as f:
    f.write("""We are observing that we have to adjust the learning rate not too big and not to small. With big learning rates we overfitt very quickly and never reach a minimum - the loss is going up and diverging then. 
With learning rates too small we have a convergance but a very slow and steady one and we seam also not to get close to an optimum. The best approach seems to be taking a learning rate of 0.001 and utilizing the early stop!
This would mean to finsih training here after about 30 epochs. While we have several nice converging training losses this one is the only one where also the test loss is accurate.""")